In [20]:
%autoreload 2
import torch
from trippletLoss.datasets import TripletAudio

K, MAX_CLOSE_NEG, MAX_FAR_NEG = 5, 15, 15
BATCH_SIZE, INPUT_D, OUTPUT_D = 1, 192, 128

triplet_train_dataset = TripletAudio(True, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_test_dataset = TripletAudio(False, K, MAX_CLOSE_NEG, MAX_FAR_NEG)
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [32]:
# Set up the network and training parameters
from trippletLoss.networks import AnchorNet, EmbeddingNet, TripletNet
from trippletLoss.losses import TripletLoss
import torch.optim as optim
from torch.optim import lr_scheduler

MARGIN, LEARNING_RATE, N_EPOCHS, LOG_INT = 0.5, 1e-3, 20, 100

#define model
anchor_net = AnchorNet(triplet_train_dataset.get_dataset(), 2, 1, True)
embedding_net = EmbeddingNet(anchor_net)
model = TripletNet(embedding_net)

loss_fn = TripletLoss(MARGIN)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

In [34]:
from trippletLoss.trainer import fit
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, N_EPOCHS, {}, LOG_INT)